In [1]:
import pandas as pd

In [2]:
path = "C:\Bi\Win Backup\LongDA\Lvl2\Class8 FIFO\FIFO"
customers = pd.read_csv(path + "\customer.csv")
documents = pd.read_csv(path + "\document.csv")
movements = pd.read_csv(path + "\movement.csv")

In [3]:
customers.rename(columns={"id": "customer_id"}, inplace=True)
customers

,customer_id,contact
0,1,boris0407@gmail.com
1,2,candywong@gmail.com
2,3,flora2002@gmail.com
3,4,glory@gmail.com
4,5,himsonfong@gmail.com
5,6,longthmax@gmail.com
6,7,quanglong@gmail.com
7,8,hung@gmail.com


In [4]:
documents['customer_id'].fillna(0, inplace = True)
documents['customer_id'] = documents['customer_id'].astype(int)
documents.rename(columns = {"id": "document_id"}, inplace=True)
documents

,document_id,customer_id,type
0,1,0,purchase
1,2,1,sales_order
2,3,2,sales_order
3,4,0,purchase
4,5,0,purchase
5,6,0,purchase
6,7,3,sales_order
7,8,1,sales_order
8,9,4,sales_order
9,10,0,purchase


In [5]:
movements['created_at'] = pd.to_datetime(movements['created_at'])
movements

,id,document_id,warehouse,sku,quantity,balance,created_at
0,1,1.0,HK,iPhone,10,10,2021-01-01
1,2,1.0,HK,iPod,3,3,2021-01-01
2,3,2.0,HK,iPod,-1,2,2021-01-02
3,4,3.0,HK,iPod,-2,0,2021-01-03
4,5,4.0,HK,iPod,5,5,2021-02-01
5,6,5.0,US,iMac,5,5,2021-02-01
6,7,5.0,US,iPhone,2,2,2021-02-01
7,8,6.0,HK,iMac,5,5,2021-02-02
8,9,7.0,HK,iPod,-4,1,2021-02-08
9,10,8.0,HK,iMac,-1,4,2021-02-09


In [10]:
def get_purchase_list():
    df = documents[documents['type'] == 'purchase'].merge(customers, how = "left", on = "customer_id")
    df['contact'] = df['contact'].fillna("Guest")
    return df[df['type'] =="purchase"].groupby('contact')['type'].count().reset_index().sort_values(by = 'type', ascending = False)

get_purchase_list()

,contact,type
0,Guest,7
3,quanglong@gmail.com,6
1,hung@gmail.com,3
2,longthmax@gmail.com,2


In [7]:
def get_HK_quantity():
    date = pd.to_datetime(input("Enter date (from 2021-01-01 to 2021-03-10): "))
    HK = movements[(movements['warehouse'] == "HK") & (movements['created_at'] <= date)]
    HK = HK.groupby(['warehouse','sku'])['quantity'].sum().reset_index().rename(columns = {"quantity": "balance"})
    HK = pd.DataFrame(HK)
    return (
        print(f"The goods of HK warehouse as of {date} is:\n"),
        print(HK)
    )
    
get_HK_quantity()

The goods of HK warehouse as of 2021-03-03 00:00:00 is:

  warehouse         sku  balance
0        HK  MacBookAir       10
1        HK  MacBookPro        3
2        HK      MacPro        4
3        HK        iMac        5
4        HK      iPhone       10
5        HK        iPod        1


(None, None)

In [8]:
def get_age_group():
    date = pd.to_datetime(input("Enter date (from 2021-01-01 to 2021-03-10): "))
    HK = movements[(movements['warehouse'] == "HK") & (movements['created_at'] <= date)]
    HK['age_of_inventory'] = date - HK['created_at']
    HK['age_group'] = HK['age_of_inventory'].apply(lambda x: "0-30 days" if x <= pd.Timedelta(days=30) else "31-60 days" if pd.Timedelta(days=30)<x<=pd.Timedelta(days=60) else "61-90 days" if pd.Timedelta(days=60)<=x<=pd.Timedelta(days=90) else "90+ days")
    HK.sort_values(by = ['sku','created_at'], ascending= True, inplace = True)
    print(f"Date of inventory: {date}")
    return HK

get_age_group()

Date of inventory: 2021-04-02 00:00:00


C:\Users\duytm\AppData\Local\Temp\ipykernel_17284\3371993262.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HK['age_of_inventory'] = date - HK['created_at']
C:\Users\duytm\AppData\Local\Temp\ipykernel_17284\3371993262.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HK['age_group'] = HK['age_of_inventory'].apply(lambda x: "0-30 days" if x <= pd.Timedelta(days=30) else "31-60 days" if pd.Timedelta(days=30)<x<=pd.Timedelta(days=60) else "61-90 days" if pd.Timedelta(days=60)<=x<=pd.Timedelta(days=90) e

,id,document_id,warehouse,sku,quantity,balance,created_at,age_of_inventory,age_group
15,16,NaN,HK,MacBookAir,10,10,2021-01-01,91 days,90+ days
16,17,NaN,HK,MacBookPro,3,3,2021-01-01,91 days,90+ days
17,18,NaN,HK,MacPro,4,4,2021-02-09,52 days,31-60 days
7,8,6.0,HK,iMac,5,5,2021-02-02,59 days,31-60 days
9,10,8.0,HK,iMac,-1,4,2021-02-09,52 days,31-60 days
11,12,10.0,HK,iMac,1,5,2021-03-02,31 days,31-60 days
12,13,11.0,HK,iMac,-1,4,2021-03-08,25 days,0-30 days
0,1,1.0,HK,iPhone,10,10,2021-01-01,91 days,90+ days
1,2,1.0,HK,iPod,3,3,2021-01-01,91 days,90+ days
2,3,2.0,HK,iPod,-1,2,2021-01-02,90 days,61-90 days


In [9]:
def calculate_FIFO():
    date = pd.to_datetime(input("Enter date (from 2021-01-01 to 2021-03-10): "))
    print(f"Date of inventory: {date}")
    movements['age_of_inventory'] = date - movements['created_at']
    movements['age_group'] = movements['age_of_inventory'].apply(lambda x: "0-30 days" if x <= pd.Timedelta(days=30) else "31-60 days" if pd.Timedelta(days=30)<x<=pd.Timedelta(days=60) else "61-90 days" if pd.Timedelta(days=60)<=x<=pd.Timedelta(days=90) else "90+ days")
    movements.sort_values(by = 'created_at', ascending= True, inplace = True)
    HK = movements[movements['warehouse'] == "HK"]
    HK.sort_values(by = ['sku','created_at'], ascending= True, inplace = True)
    FF = HK.pivot_table(index = 'sku', columns = 'age_group', values = 'quantity', aggfunc = 'sum').fillna(0).reset_index()
    FF['total_balance'] = FF['0-30 days']+ FF['31-60 days']+ FF['61-90 days']+ FF['90+ days']
    FF['90+'] = FF.apply(lambda rows: 0 if rows['total_balance'] == 0 else (
        rows['total_balance'] if rows['90+ days'] > rows['total_balance'] else (
            rows['90+ days'])),axis =1)
    FF['61-90'] = FF.apply(lambda rows: 0 if (rows['total_balance']- rows['90+']) == 0 else(
        rows['61-90 days'] if rows['61-90 days'] == rows['total_balance'] else (
            rows['total_balance'] - rows['90+'] -abs(rows['31-60 days'] + rows['0-30 days']))), axis =1)
    FF['31-60'] = FF.apply(lambda rows: 0 if (rows['total_balance'] - rows['90+'] - rows['61-90']) == 0 else (
        (rows['total_balance'] - rows['90+'] - rows['61-90'])), axis = 1)
    FF['0-30'] = FF['total_balance'] - FF['90+'] - FF['61-90'] - FF['31-60']
    FF.drop(columns=['0-30 days','31-60 days','61-90 days','90+ days'], inplace=True)
    return FF

calculate_FIFO() 

Date of inventory: 2021-04-02 00:00:00


C:\Users\duytm\AppData\Local\Temp\ipykernel_17284\2335958417.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HK.sort_values(by = ['sku','created_at'], ascending= True, inplace = True)


age_group,sku,total_balance,90+,61-90,31-60,0-30
0,MacBookAir,10.0,10.0,0.0,0.0,0.0
1,MacBookPro,3.0,3.0,0.0,0.0,0.0
2,MacPro,4.0,0.0,0.0,4.0,0.0
3,iMac,4.0,0.0,0.0,4.0,0.0
4,iPhone,10.0,10.0,0.0,0.0,0.0
5,iPod,0.0,0.0,0.0,0.0,0.0
